# Active contour segmentation pre-processing performance comparison

In [1]:
# Import libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import utils
import lsm
from imageio import imwrite
from skimage import img_as_float
from scipy.ndimage.filters import gaussian_filter # For a Gaussian blur

## Define hyperparameters

In [2]:
# Define the size
n1 = 1024
n2 = 1024

# Paramters for a level set method
rho = 1        # No need to modify this. It doesn't affect the result very much.
sigma   = 1e-7 # A parameter for a convolution. Higher the value, lesser the noise and lower the accuracy.
omega   = 0.2  # A ratio of gG and gL functions. gL + omega * gG.
epsilon = 0.2  # A paramter for a H_epsilon function.

# Run a method
max_iter = 600 # Maximum iterations
print_every = -1 # Verbose. Print every [print_every] iteration. It won't print out if -1
step_size = 500 # step size of the method

## ENTER YOUR INDICES HERE

Split your total workload between notebooks for multiprocessing.
Divide according to your available cpu.

In [3]:
start_indice = 0
end_indice = 4
data_dir = "../../unannotated_data/"    # directory of the unannotated image files
output_dir = "./segmentation_data/"     # directory you want the segmentations to be stored

## Import data

In [4]:
# Get Data
data = utils.obtain_indicated_data(s_i=start_indice, e_i=end_indice, data_dir=data_dir)
names = data['names']
images = data['images']

## Comparison of Gaussian blur and Gaussian blur + contrast enhancement

In [5]:
def preprocess_img(img_index, sigmas=[1.0, 1.5, 2.0], clip_limits=[2.0, 4.0]):
    
    # Resize the image
    image = images[img_index]
    image = cv2.resize(image, (n2, n1))  # Resizing to (n2,n1)
    name = names[img_index]
    
    # Three sigmas and two clip limits -> 6 total segmentations
    processed_images = []
    for sigma in sigmas:
        
        # Apply Gaussian blur
        gb_image = utils.scale_to_uint(gaussian_filter(image, sigma=sigma))
        
        for clip_limit in clip_limits:
            # Improve contrast
            clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(8, 8))
            processed_images.append(img_as_float(np.array(clahe.apply(np.expand_dims(gb_image, axis=-1))).squeeze()))
            
    return image, name, processed_images
                                    
# Create a output directory if it doesn't exist already
if not os.path.exists(output_dir): os.mkdir(output_dir)
                                    
combos = [(1.0, 2.0), (1.0, 4.0), (1.5, 2.0), (1.5, 4.0), (2.0, 2.0), (2.0, 4.0)]
num_imgs = len(names)
total_jobs = num_imgs * len(combos)

# Perform segmentations
for n, i in enumerate(range(start_indice, end_indice)):
    print("Starting image with index: {}".format(i))
    image, name, processed_imgs = preprocess_img(n)
    
    # Make a directory for each image file
    name = os.path.splitext(name)[0]
    dir_name = output_dir + name + "/"
    if not os.path.exists(dir_name): os.mkdir(dir_name)
        
    # Save original file so we can easily compare within the directory
    imwrite(dir_name + name + ".png", (image * 255).astype(np.uint8))
    
    for j, img in enumerate(processed_imgs):
        # Create a lsm object
        curr_lsm = lsm.LSM(n1, n2, rho, sigma, omega, epsilon)

        # Initialize based on center of mass using smoothing method
        curr_lsm.initialize_phi(image)

        # Run a method.
        phi = utils.run_lsm(img, curr_lsm, step_size=step_size, max_iter=max_iter, 
                            print_every=print_every, do_plot=False)
        
        # Obtain segmentation
        segment = np.zeros_like(image)
        segment[phi >= 0] = 1
        segment = (segment * 255).astype(np.uint8)
        
        # Later the chosen file names can have these parameters stripped easily with a script
        imwrite(dir_name + name + ".sigma" + str(combos[j][0]) + "_clip" + str(combos[j][1]) + ".png", segment)
        
        print("{:.2f}% completed".format(100*(1+j+i*6)/total_jobs))
        
print("Finished")

Starting image with index: 0
4.17% completed
8.33% completed
12.50% completed
16.67% completed
20.83% completed
25.00% completed
Starting image with index: 1
29.17% completed
33.33% completed
37.50% completed
41.67% completed
45.83% completed
50.00% completed
Starting image with index: 2
54.17% completed
58.33% completed
62.50% completed
66.67% completed
70.83% completed
75.00% completed
Starting image with index: 3
79.17% completed
83.33% completed
87.50% completed
91.67% completed
95.83% completed
100.00% completed
Finished
